#### Step 1: fclass processing

First prepare POIS (points - the code below is a modular script for fclass processing), streets (lines, processed with simplify, planarize, sDNA prepare network, test it out with sDNA integral analysis to check that its ready), POP (polygons)


In [ ]:
import arcpy
import os

In [ ]:
#Combine different fields into one fclass within a shp file - for Leisure.geojson converted to point shp

shp_path = r"D:\UserData16\heyutian\data\01_Reference\20250712_Boston_POIS\original\Leisure-POINT.shp"

# Define fields to check in order of priority
field_priority = ["amenity", "leisure", "tourism"]  # Extend this list as needed
output_field = "fclass"

# --- Add 'fclass' field if it doesn't exist ---
existing_fields = [f.name for f in arcpy.ListFields(shp_path)]
if output_field not in existing_fields:
    arcpy.AddField_management(shp_path, output_field, "TEXT")

# --- Build the expression ---
# Pass all fields into the reclass function
expression_fields = ", ".join([f"!{f}!" for f in field_priority])
expression = f"reclass({expression_fields})"

# --- Define the code block to check each field in order ---
# Accepts variable number of arguments using *args
code_block = """
def reclass(*args):
    for val in args:
        if val is not None and str(val).strip():
            return str(val).lower().replace(' ', '_')
    return "unknown"
"""

# --- Calculate field using expression and code block ---
arcpy.CalculateField_management(
    in_table=shp_path,
    field=output_field,
    expression=expression,
    expression_type="PYTHON3",
    code_block=code_block
)

print(f"fclass calculated using priority fields: {field_priority}")

# Optional post-processing: Update unknowns based on 'sport'
with arcpy.da.UpdateCursor(shp_path, ["sport", "fclass"]) as cursor:
    for row in cursor:
        sport_val, fclass_val = row
        if fclass_val == "unknown" and sport_val is not None and str(sport_val).strip():
            row[1] = "sports_centre"
            cursor.updateRow(row)

print("Updated fclass to 'sports_centre' where sport exists and fclass was 'unknown'")



In [ ]:
# modular script for fclass/merge/clip with library config


input_folder = r"D:\UserData16\heyutian\data\01_Reference\20250712_Boston_POIS\original"
output_folder = r"D:\UserData16\heyutian\data\01_Reference\20250712_Boston_POIS\processed"
clip_boundary = r"Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_boundary_edi_Dissolve"
os.makedirs(output_folder, exist_ok=True)

# === CONFIGURATION ===
# For each shapefile in input_folder, specify one action:
# - "full" = add fclass field, calculate it, then clip
# - "clip_only" = just clip (skip field calc)
#   "merge_only"  – Do not process or clip, just merge existing file
# - "skip" = ignore this file completely

file_configs = {
    "boston_POIS_work_select.shp": {"action": "merge_only"},
    "PLACES_OF_WORSHIP_PT.shp":     {"action": "full","fixed_fclass": None,"source_fields": ["TYPE"]},

# example syntax below for various configs:
#     "CHILDCARE_PT.shp":      {"action": "full","fixed_fclass": "childcare", "source_fields": None},
#     "childcare.shp":     {"action": "full","fixed_fclass": None,"source_fields": ["CATEGORY"]},
#     "DCRPOOLS_PT.shp": {"action": "clip_only"},
#     "leisure_processed.shp": {"action": "merge_only"},  # Already clipped
# Add more shapefiles here
}

# Helper: build CalculateField expression + code block
def build_calc_expr_code(fields):
    expr = f"reclass({', '.join(['!' + f + '!' for f in fields])})"
    code_blk = """
def reclass(*args):
    for val in args:
        if val is not None and str(val).strip():
            return str(val).lower().replace(' ', '_')
    return 'unknown'
"""
    return expr, code_blk

# Collect clipped shapefiles for merging
clipped_files = []
for fname in os.listdir(input_folder):
    if not fname.lower().endswith(".shp"):
        continue

    if fname not in file_configs:
        print(f"Skipping {fname} – not in config")
        continue

    config = file_configs[fname]
    action = config.get("action", "skip")

    in_path = os.path.join(input_folder, fname)
    base_name = os.path.splitext(fname)[0]
    out_path = os.path.join(output_folder, f"{base_name}_processed.shp")

    if action == "skip":
        print(f"Skipping {fname} per config")
        continue

    if action == "merge_only":
        print(f"Adding {fname} to merge list only (no processing)")
        clipped_files.append(in_path)
        continue

    if os.path.exists(out_path):
        print(f"Output for {fname} already exists, skipping processing")
        clipped_files.append(out_path)
        continue

    print(f"Processing {fname} with action '{action}'")

    if action == "full":
        # Ensure fclass field exists
        existing_fields = [f.name for f in arcpy.ListFields(in_path)]
        if "fclass" not in existing_fields:
            arcpy.AddField_management(in_path, "fclass", "TEXT")

        fixed = config.get("fixed_fclass")
        source = config.get("source_fields")

        if fixed:
            with arcpy.da.UpdateCursor(in_path, ["fclass"]) as cursor:
                for row in cursor:
                    row[0] = fixed
                    cursor.updateRow(row)
            print(f"Set fixed fclass '{fixed}' for all features")

        elif source:
            expr, code_blk = build_calc_expr_code(source)
            arcpy.CalculateField_management(in_path, "fclass", expr, "PYTHON3", code_blk)
            print(f"Calculated fclass from fields: {source}")

        else:
            print(f"No fclass rule specified, skipping classification")

    if action in ("full", "clip_only"):
        arcpy.Clip_analysis(in_path, clip_boundary, out_path)
        clipped_files.append(out_path)
        print(f"Clipped to: {out_path}")

# final merge
if clipped_files:
    merged_output = os.path.join(output_folder, "boston_POIS_educivic.shp")
    print(f"Merging {len(clipped_files)} shapefiles into {merged_output}...")
    arcpy.Merge_management(clipped_files, merged_output)
    print("Merge complete.")

    # --- Deduplication ---
    # Count features before
    count_before = int(arcpy.GetCount_management(merged_output)[0])

    # Delete exact duplicate points with same fclass
    arcpy.DeleteIdentical_management(
        in_dataset=merged_output,
        fields=["Shape","fclass"],              # Consider adding more fields if needed
        xy_tolerance="0 Meters",
    )

    # Count features after
    count_after = int(arcpy.GetCount_management(merged_output)[0])
    num_deleted = count_before - count_after

    print(f"Removed {num_deleted} duplicate point(s) from merged shapefile.")
    print(f"Final output saved at: {merged_output}")

else:
    print("No shapefiles were processed or found to merge.")


In [ ]:
# clean up attribute fields that are null or empty (useful for geojsons scraped from overpass turbo)

import arcpy

shp_path = r"D:\UserData16\heyutian\data\01_Reference\20250712_Boston_POIS\processed\boston_POIS_educivic.shp"

# identify fields with null or blank values

empty_fields = []

# List all fields except OID and geometry
fields = [f.name for f in arcpy.ListFields(shp_path) if f.type not in ("OID", "Geometry")]

for field in fields:
    has_data = False
    with arcpy.da.SearchCursor(shp_path, [field]) as cursor:
        for row in cursor:
            value = row[0]
            if value not in [None, "", " "]:
                has_data = True
                break
    if not has_data:
        empty_fields.append(field)

print("Fields that are empty across all rows:")
for f in empty_fields:
    print("   -", f)

# delete those empty rows
if empty_fields:
    arcpy.DeleteField_management(shp_path, empty_fields)
    print(f"🧹 Deleted {len(empty_fields)} empty fields.")
else:
    print("No empty fields to delete.")

#### Step 2: Spatial joins - road with POIS
Spatial join road network with each POIS (cumulatively), within a distance, 400m. Create new column, join count = pois_shop (this is the POIS density). Then spatial join this resulting file with point data files POIS work, POIS educivic, etc. end with 1 file with 4 pois columns
Kaunas_streets_good with Kaunas_POIS_work, then the result with Kaunas_POIS_leisure, etc
End with 1 file: Kaunas_st_allPOIS, with 4 columns of 4 POIS densities

In [ ]:
import arcpy
import os
from itertools import product

# define file paths
input_folder = r"D:\UserData16\heyutian\data\01_Reference\20250730_Boston_spatialjoins\input"
output_folder = r"D:\UserData16\heyutian\data\01_Reference\20250730_Boston_spatialjoins\output"

# find files
streets = [f for f in os.listdir(input_folder) if f.startswith("boston_streets") and f.endswith(".shp")]
POIS = [f for f in os.listdir(input_folder) if f.startswith("boston_POIS") and f.endswith(".shp")]

# Use the first matching streets file
base_streets_file = streets[0]
current_streets_path = os.path.join(input_folder, base_streets_file)

pois_density_fields = []

def build_nullable_fieldmap(target_fc, join_fc):
    field_mappings = arcpy.FieldMappings()
    # Add all fields from the streets (target) layer only
    field_mappings.addTable(target_fc)
    # Do NOT add any fields from join_fc (POIS)
    # Join_Count field will be auto-generated by SpatialJoin
    return field_mappings

# Track cumulative suffix for naming
cumulative_suffix = ""

# Loop through POI files and join each cumulatively
for i, poi_file in enumerate(POIS):
    poi_path = os.path.join(input_folder, poi_file)
    poi_suffix = poi_file.split("boston_POIS_")[-1].replace(".shp", "")
    cumulative_suffix += f"_{poi_suffix}"

    gdb_path = r"Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb"
    out_name = f"{os.path.splitext(base_streets_file)[0]}{cumulative_suffix}"
    out_path = os.path.join(gdb_path, out_name)


    print(f"Joining {os.path.basename(current_streets_path)} with {poi_file} → {out_name}")

    # Prepare field map
    field_map = build_nullable_fieldmap(current_streets_path, poi_path)

    # Perform spatial join
    arcpy.analysis.SpatialJoin(
        target_features=current_streets_path,
        join_features=poi_path,
        out_feature_class=out_path,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_map,
        match_option="WITHIN_A_DISTANCE",
        search_radius="400 Meters"
    )

    print(f"Saved: {out_path}")

    # Add POIS density field
    # Truncate field name to max 10 chars (safe for shapefiles)
    density_field = (f"p_{poi_suffix.lower()}")[:10]
    # Add field with truncated name, but human-readable alias
    arcpy.AddField_management(out_path, density_field, "DOUBLE", field_alias=f"POIS_{poi_suffix}")

    pois_density_fields.append(density_field)


    # Determine the join count field name
    fields = [f.name for f in arcpy.ListFields(out_path)]
    join_count_field = next((f for f in fields if f.lower().startswith("join_cou")), None)

    if join_count_field:
        arcpy.CalculateField_management(out_path, density_field, f"!{join_count_field}!", "PYTHON3")
        print(f"Calculated density field: {density_field} from {join_count_field}")
        # Delete the Join_Count field right after
        arcpy.DeleteField_management(out_path, join_count_field)
        print(f"Deleted intermediate Join_Count field: {join_count_field}")

    else:
        print("Warning: Join_Count field not found!")


    # Update current path to new output for next cumulative join
    current_streets_path = out_path

# After all joins are done
all_fields = [f.name for f in arcpy.ListFields(current_streets_path)]

# Fields to keep: Shape + fclass + POIS Density Fields
keep_fields = ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area','fclass'] + pois_density_fields

# Determine fields to delete
delete_fields = [f for f in all_fields if f not in keep_fields and f.lower() != 'shape']

# Delete them in bulk
if delete_fields:
    arcpy.DeleteField_management(current_streets_path, delete_fields)
    print(f"🧹 Final cleanup: Deleted fields → {delete_fields}")
else:
    print("No extra fields to delete.")



Joining boston_streets_good.shp with boston_POIS_educivic.shp → boston_streets_good_educivic
Saved: Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_streets_good_educivic
Calculated density field: p_educivic from Join_Count
Deleted intermediate Join_Count field: Join_Count
Joining boston_streets_good_educivic with boston_POIS_leisure.shp → boston_streets_good_educivic_leisure
Saved: Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_streets_good_educivic_leisure
Calculated density field: p_leisure from Join_Count
Deleted intermediate Join_Count field: Join_Count
Joining boston_streets_good_educivic_leisure with boston_POIS_shopping.shp → boston_streets_good_educivic_leisure_shopping
Saved: Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_streets_good_educivic_leisure_shopping
Calculated density field: p_shopping from Join_Count
Deleted intermediate Join_Count field: Join_Count
Joining boston_streets_good_educivic_leisure_shopping with boston_POIS_work.shp → boston_streets_good_educ

#### Step 3: sDNA analysis
(new step) run SDNA analysis with input polyline features from the previous step i.e. with all 4 POIS densities, but weighted by 4 different POIS
e.g. 1k x 4 POIS. Unlike the previous step which requires the output to be inside gdb, this step for sDNA toolbox to work the input polyline file has to be outside of a gdb.
"Integral Analysis" -> check "betweenness is bidirectional", radius = 1000; check "continuous space", run
Destination weight


In [ ]:
import arcpy
import os

# Paths
input_polyline = r"D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd\boston_st_gd_AllPOIS.shp"
output_folder = r"D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd"

# List of your POIS density fields (adjust names exactly as in attribute table)
pois_density_fields = ['p_educivic', 'p_leisure', 'p_shopping', 'p_work']  # example names

# Import sDNA toolbox with alias 'sdna'
sdna_toolbox_path = r"Z:\heyutian\sDNA\sDNA.pyt"
arcpy.ImportToolbox(sdna_toolbox_path, "sdna")

# Ensure output folder exists
if not arcpy.Exists(output_folder):
    arcpy.CreateFolder_management(os.path.dirname(output_folder), os.path.basename(output_folder))

# Create a Feature Layer from the input polyline feature class
input_layer = "input_streets_layer"
arcpy.MakeFeatureLayer_management(input_polyline, input_layer)

for weight_field in pois_density_fields:
    print(f"Running sDNA Integral analysis weighted by {weight_field}...")

    # Define output path
    out_name = f"boston_sDNA1k_{weight_field}"
    out_path = os.path.join(output_folder, out_name)

    # Run the Integral Analysis tool with your parameters
    arcpy.sdna.sDNAIntegral(
    input=input_layer,
    output=out_path,
    betweenness=True,
    bidir=True,
    junctions=False,
    hull=False,
    start_gs=None,
    end_gs=None,
    analmet="EUCLIDEAN",
    radii="1000",
    bandedradii=False,
    cont=True,
    radmet="EUCLIDEAN",
    weighting="Link",
    origweight=None,
    destweight=weight_field,
    zonefiles=None,
    odfile=None,
    custommetric=None,
    disable="",
    oneway=None,
    intermediates="",
    advanced=""
    )

    print(f"Saved sDNA Integral output to {out_path}")


Running sDNA Integral analysis weighted by p_educivic...
Saved sDNA Integral output to D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd\boston_sDNA1k_p_educivic
Running sDNA Integral analysis weighted by p_leisure...
Saved sDNA Integral output to D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd\boston_sDNA1k_p_leisure
Running sDNA Integral analysis weighted by p_shopping...
Saved sDNA Integral output to D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd\boston_sDNA1k_p_shopping
Running sDNA Integral analysis weighted by p_work...
Saved sDNA Integral output to D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd\boston_sDNA1k_p_work


#### Step 4: Spatial joins - NQPD combination then with population
Rename the NQPD column to each POI, e.g. nqpd1kwork, before each join, so that they remain discrete. Spatial join the 4 sDNA_POIS_work etc files cumulatively, such that there are 4 columns. Keep default spatial join settings (i.e. intersect, no meters defined).
Spatial join network with POP with 30m radius
End result will be 5 columns: POP, nqpd1kwork, nqpd1klsr, nqpd1kshop, nqpd1kedcv

In [ ]:
# import arcpy
# import os

# # Paths
# input_shp_folder = r"D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd"
# gdb_path = r"Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb"

# # Step 1: Copy shapefiles to GDB as feature classes
# shp_files = [f for f in os.listdir(input_shp_folder) if f.endswith(".shp")]

# for shp in shp_files:
#     shp_path = os.path.join(input_shp_folder, shp)
#     out_name = os.path.splitext(shp)[0]
#     out_fc = os.path.join(gdb_path, out_name)

#     # Copy shapefile to gdb as feature class
#     arcpy.FeatureClassToFeatureClass_conversion(shp_path, gdb_path, out_name)
#     print(f"Copied {shp} to {out_fc}")

# # Step 2: Set arcpy workspace to gdb for easy listing of feature classes
# arcpy.env.workspace = gdb_path

# # List feature classes in GDB starting with sDNA prefix
# sdna_fcs = arcpy.ListFeatureClasses("boston_sDNA1k_p_*")

# # Use first file as base for cumulative join
# base_fc = sdna_fcs[0]
# cumulative_fc = "temp_cumulative"
# arcpy.CopyFeatures_management(base_fc, cumulative_fc)

# # Process each sDNA feature class
# for i, sdna_fc in enumerate(sdna_fcs):
#     suffix = sdna_fc.split("boston_sDNA1k_p_")[-1][:4]
#     nqpd_field_new = f"nqpd1k{suffix}"

#     # Rename NQPD field inside the feature class
#     fields = arcpy.ListFields(sdna_fc)
#     nqpd_field = next((f.name for f in fields if f.name.upper().startswith("NQPD")), None)

#     if nqpd_field:
#         arcpy.AlterField_management(sdna_fc, nqpd_field, new_field_name=nqpd_field_new)
#         print(f"Renamed field {nqpd_field} to {nqpd_field_new} in {sdna_fc}")
#     else:
#         print(f"NQPD field not found in {sdna_fc}")

#     # Skip cumulative join for base file (already copied)
#     if i == 0:
#         continue

#     # Cumulative spatial join - output a new FC each time
#     out_temp = f"temp_cumulative_{i}"
#     arcpy.analysis.SpatialJoin(
#         target_features=cumulative_fc,
#         join_features=sdna_fc,
#         out_feature_class=out_temp,
#         join_operation="JOIN_ONE_TO_ONE",
#         join_type="KEEP_ALL",
#         match_option="INTERSECT"
#     )
#     print(f"Joined {sdna_fc} into cumulative result → {out_temp}")
#     cumulative_fc = out_temp

# # Final Spatial Join with POP — FieldMappings Setup
# pop_layer = r"D:\UserData16\heyutian\data\00_Clean\Boston_data\boston_pop_blocks.shp"
# output_final = r"D:\UserData16\heyutian\data\01_Reference\20250804_2_nqpd1k_boston_st_gd\boston_nqpf1k_allPOIS_stgd.shp"

# # Build FieldMappings cleanly
# field_mappings = arcpy.FieldMappings()

# fields_in_cumulative = [f.name for f in arcpy.ListFields(cumulative_fc)]
# print("Fields in cumulative_fc:", fields_in_cumulative)

# for field_name in ['nqpd1kwork', 'nqpd1kleis', 'nqpd1kshop', 'nqpd1keduc']:
#     if field_name not in fields_in_cumulative:
#         print(f"WARNING: Field '{field_name}' not found in cumulative_fc!")
#     else:
#         fmap = arcpy.FieldMap()
#         fmap.addInputField(cumulative_fc, field_name)
#         fmap.outputField.name = field_name
#         fmap.outputField.aliasName = field_name
#         field_mappings.addFieldMap(fmap)

# # Add all NQPD fields (they now exist in cumulative_fc)
# for field_name in ['nqpd1kwork', 'nqpd1kleis', 'nqpd1kshop', 'nqpd1keduc']:
#     fmap = arcpy.FieldMap()
#     fmap.addInputField(cumulative_fc, field_name)
#     fmap.outputField.name = field_name
#     fmap.outputField.aliasName = field_name
#     field_mappings.addFieldMap(fmap)

# # Add POP20 from pop_layer
# fmap_pop = arcpy.FieldMap()
# fmap_pop.addInputField(pop_layer, 'POP20')
# fmap_pop.outputField.name = 'POP20'
# fmap_pop.outputField.aliasName = 'POP20'
# field_mappings.addFieldMap(fmap_pop)

# # Perform Spatial Join with POP20, keeping only desired fields
# arcpy.SpatialJoin_analysis(
#     target_features=cumulative_fc,
#     join_features=pop_layer,
#     out_feature_class=output_final,
#     join_operation="JOIN_ONE_TO_ONE",
#     join_type="KEEP_ALL",
#     match_option="WITHIN_A_DISTANCE",
#     search_radius="30 Meters",
#     field_mapping=field_mappings
# )
# print(f"Final network with POP joined → {output_final}")

# # Cleanup: Delete all intermediate temp_cumulative_* feature classes
# arcpy.env.workspace = gdb_path
# temp_fcs = arcpy.ListFeatureClasses("temp_cumulative*")

# for temp_fc in temp_fcs:
#     arcpy.Delete_management(temp_fc)
#     print(f"Deleted intermediate feature class: {temp_fc}")

# print("Cleanup complete!")


Copied boston_sDNA1k_p_educivic.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_educivic
Copied boston_sDNA1k_p_leisure.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_leisure
Copied boston_sDNA1k_p_shopping.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_shopping
Copied boston_sDNA1k_p_work.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_work
Copied boston_st_gd_AllPOIS.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_st_gd_AllPOIS
Renamed field NQPDE1000c to nqpd1keduc in boston_sDNA1k_p_educivic
Renamed field NQPDE1000c to nqpd1kleis in boston_sDNA1k_p_leisure
Joined boston_sDNA1k_p_leisure into cumulative result → temp_cumulative_1
Renamed field NQPDE1000c to nqpd1kshop in boston_sDNA1k_p_shopping
Joined boston_sDNA1k_p_shopping into cumulative result → temp_cumulative_2
Renamed field NQPDE1000c to nqpd1kwork in boston_sDNA1k_p_work
Joined boston_sDNA1k_p_work into cumulative result → temp_cumulati

RuntimeError: FieldMap: Error in adding input field to field map

In [ ]:
# import arcpy
# import os

# # Paths
# input_shp_folder = r"D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd"
# gdb_path = r"Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb"
# pop_layer = r"D:\UserData16\heyutian\data\00_Clean\Boston_data\boston_pop_blocks.shp"
# output_final = r"D:\UserData16\heyutian\data\01_Reference\20250804_2_nqpd1k_boston_st_gd\boston_nqpf1k_allPOIS_stgd.shp"

# # Step 1: Copy shapefiles to GDB as feature classes
# shp_files = [f for f in os.listdir(input_shp_folder) if f.endswith(".shp")]

# for shp in shp_files:
#     shp_path = os.path.join(input_shp_folder, shp)
#     out_name = os.path.splitext(shp)[0]
#     arcpy.FeatureClassToFeatureClass_conversion(shp_path, gdb_path, out_name)
#     print(f"Copied {shp} to {out_name}")

# arcpy.env.workspace = gdb_path

# # Step 2: Rename NQPD fields uniquely in each POI feature class
# sdna_fcs = arcpy.ListFeatureClasses("boston_sDNA1k_p_*")
# renamed_fcs = []

# for fc in sdna_fcs:
#     suffix = fc.split("boston_sDNA1k_p_")[-1][:4]
#     new_field_name = f"nqpd1k{suffix}"

#     fields = arcpy.ListFields(fc)
#     nqpd_field = next((f.name for f in fields if f.name.upper().startswith("NQPD")), None)
#     if nqpd_field:
#         arcpy.AlterField_management(fc, nqpd_field, new_field_name=new_field_name)
#         print(f"Renamed field {nqpd_field} to {new_field_name} in {fc}")
#         renamed_fcs.append((fc, new_field_name))
#     else:
#         print(f"No NQPD field found in {fc}")

# # Step 3: Copy base network for join target
# base_fc = sdna_fcs[0]  # or your preferred network FC
# network_fc = "network_base"
# arcpy.CopyFeatures_management(base_fc, network_fc)

# # Step 4: For each POI FC, spatial join to network, output separate join FCs
# joined_fcs = []

# for fc, nqpd_field in renamed_fcs:
#     out_join_fc = f"{fc}_joined"
#     arcpy.analysis.SpatialJoin(
#         target_features=network_fc,
#         join_features=fc,
#         out_feature_class=out_join_fc,
#         join_operation="JOIN_ONE_TO_ONE",
#         join_type="KEEP_ALL",
#         match_option="INTERSECT"
#     )
#     print(f"Joined {fc} → {out_join_fc}")
#     joined_fcs.append((out_join_fc, nqpd_field))

# # Step 5: Iteratively join each POI join FC back to network by unique ID (assumes 'OBJECTID' or similar unique key)
# # We'll do attribute joins here to add each NQPD field to network_fc

# # Use a unique ID field present in all feature classes
# unique_id_field = "OBJECTID"  # Change if your ID field differs

# for join_fc, nqpd_field in joined_fcs:
#     temp_joined = "network_with_" + nqpd_field
#     arcpy.JoinField_management(
#         in_data=network_fc,
#         in_field=unique_id_field,
#         join_table=join_fc,
#         join_field=unique_id_field,
#         fields=[nqpd_field]
#     )
#     print(f"Joined field {nqpd_field} from {join_fc} into {network_fc}")

# # Step 6: Spatial join POP data with field mapping for just POP20 and existing fields
# field_mappings = arcpy.FieldMappings()

# # Add all fields from network_fc except shape (you can customize this list)
# for field in arcpy.ListFields(network_fc):
#     if field.type != 'Geometry':
#         fmap = arcpy.FieldMap()
#         fmap.addInputField(network_fc, field.name)
#         fmap.outputField.name = field.name
#         fmap.outputField.aliasName = field.aliasName
#         field_mappings.addFieldMap(fmap)

# # Add POP20 field from pop_layer
# fmap_pop = arcpy.FieldMap()
# fmap_pop.addInputField(pop_layer, 'POP20')
# fmap_pop.outputField.name = 'POP20'
# fmap_pop.outputField.aliasName = 'POP20'
# field_mappings.addFieldMap(fmap_pop)

# # Perform spatial join with POP
# arcpy.analysis.SpatialJoin(
#     target_features=network_fc,
#     join_features=pop_layer,
#     out_feature_class=output_final,
#     join_operation="JOIN_ONE_TO_ONE",
#     join_type="KEEP_ALL",
#     match_option="WITHIN_A_DISTANCE",
#     search_radius="30 Meters",
#     field_mapping=field_mappings
# )
# print(f"Final spatial join with POP done: {output_final}")


Copied boston_sDNA1k_p_educivic.shp to boston_sDNA1k_p_educivic
Copied boston_sDNA1k_p_leisure.shp to boston_sDNA1k_p_leisure
Copied boston_sDNA1k_p_shopping.shp to boston_sDNA1k_p_shopping
Copied boston_sDNA1k_p_work.shp to boston_sDNA1k_p_work
Copied boston_st_gd_AllPOIS.shp to boston_st_gd_AllPOIS
Renamed field NQPDE1000c to nqpd1keduc in boston_sDNA1k_p_educivic
Renamed field NQPDE1000c to nqpd1kleis in boston_sDNA1k_p_leisure
Renamed field NQPDE1000c to nqpd1kshop in boston_sDNA1k_p_shopping
Renamed field NQPDE1000c to nqpd1kwork in boston_sDNA1k_p_work
Joined boston_sDNA1k_p_educivic → boston_sDNA1k_p_educivic_joined
Joined boston_sDNA1k_p_leisure → boston_sDNA1k_p_leisure_joined
Joined boston_sDNA1k_p_shopping → boston_sDNA1k_p_shopping_joined
Joined boston_sDNA1k_p_work → boston_sDNA1k_p_work_joined
Joined field nqpd1keduc from boston_sDNA1k_p_educivic_joined into network_base
Joined field nqpd1kleis from boston_sDNA1k_p_leisure_joined into network_base
Joined field nqpd1kshop 

In [ ]:
import arcpy
import os

# Paths
input_shp_folder = r"D:\UserData16\heyutian\data\01_Reference\20250804_sDNA_POIS_Boston_st_gd"
gdb_path = r"Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb"
pop_layer = r"D:\UserData16\heyutian\data\00_Clean\Boston_data\boston_pop_blocks.shp"
output_final = r"D:\UserData16\heyutian\data\01_Reference\20250804_2_nqpd1k_boston_st_gd\boston_nqpf1k_allPOIS_stgd.shp"

# Step 1: Copy shapefiles to GDB as feature classes
shp_files = [f for f in os.listdir(input_shp_folder) if f.endswith(".shp")]
for shp in shp_files:
    shp_path = os.path.join(input_shp_folder, shp)
    out_name = os.path.splitext(shp)[0]
    out_fc = os.path.join(gdb_path, out_name)
    arcpy.FeatureClassToFeatureClass_conversion(shp_path, gdb_path, out_name)
    print(f"Copied {shp} to {out_fc}")

# Step 2: Set arcpy workspace to gdb
arcpy.env.workspace = gdb_path

# Identify base network geometry (e.g., streets)
base_fc = "boston_st_gd_AllPOIS"  # Assuming this is the network base layer

# List sDNA feature classes (assuming naming boston_sDNA1k_p_*)
sdna_fcs = arcpy.ListFeatureClasses("boston_sDNA1k_p_*")

# Step 3: For each sDNA fc, create new NQPD field by copying from original
for sdna_fc in sdna_fcs:
    suffix = sdna_fc.split("boston_sDNA1k_p_")[-1][:4]
    new_nqpd_field = f"nqpd1k{suffix}"

    fields = arcpy.ListFields(sdna_fc)
    orig_nqpd_field = next((f.name for f in fields if f.name.upper().startswith("NQPD")), None)

    if orig_nqpd_field is None:
        print(f"Original NQPD field not found in {sdna_fc}. Skipping field creation.")
    else:
        if new_nqpd_field not in [f.name for f in fields]:
            arcpy.AddField_management(sdna_fc, new_nqpd_field, "DOUBLE")
            print(f"Added new field {new_nqpd_field} in {sdna_fc}")
        arcpy.CalculateField_management(sdna_fc, new_nqpd_field, f"!{orig_nqpd_field}!", "PYTHON3")
        print(f"Copied data from {orig_nqpd_field} to {new_nqpd_field} in {sdna_fc}")

# Step 4: Build FieldMappings combining all sDNA NQPD fields
field_mappings = arcpy.FieldMappings()

# Add all fields from base_fc (geometry & IDs)
base_fields = arcpy.ListFields(base_fc)
for field in base_fields:
    if field.type not in ["Geometry"]:
        fmap = arcpy.FieldMap()
        fmap.addInputField(base_fc, field.name)
        field_mappings.addFieldMap(fmap)

# Add the new NQPD fields from each sDNA layer
for sdna_fc in sdna_fcs:
    suffix = sdna_fc.split("boston_sDNA1k_p_")[-1][:4]
    nqpd_field = f"nqpd1k{suffix}"
    fmap = arcpy.FieldMap()
    fmap.addInputField(sdna_fc, nqpd_field)
    fmap.outputField.name = nqpd_field
    fmap.outputField.aliasName = nqpd_field
    field_mappings.addFieldMap(fmap)

# Perform spatial join of all sDNA layers into base network geometry
combined_sDNA_output = "network_with_nqpd"
arcpy.analysis.SpatialJoin(
    target_features=base_fc,
    join_features=sdna_fcs,
    out_feature_class=combined_sDNA_output,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="INTERSECT",
    field_mapping=field_mappings
)
print(f"Combined sDNA layers into {combined_sDNA_output}")

# Step 5: Spatial Join with POP (Final Join)
final_field_mappings = arcpy.FieldMappings()

# Keep all existing fields from combined_sDNA_output
for field in arcpy.ListFields(combined_sDNA_output):
    if field.type not in ["Geometry"]:
        fmap = arcpy.FieldMap()
        fmap.addInputField(combined_sDNA_output, field.name)
        final_field_mappings.addFieldMap(fmap)

# Add POP20 field from pop_layer
fmap_pop = arcpy.FieldMap()
fmap_pop.addInputField(pop_layer, 'POP20')
fmap_pop.outputField.name = 'POP20'
fmap_pop.outputField.aliasName = 'POP20'
final_field_mappings.addFieldMap(fmap_pop)

# Execute final spatial join
arcpy.analysis.SpatialJoin(
    target_features=combined_sDNA_output,
    join_features=pop_layer,
    out_feature_class=output_final,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="WITHIN_A_DISTANCE",
    search_radius="30 Meters",
    field_mapping=final_field_mappings
)
print(f"Final network with POP joined → {output_final}")

print("All done!")


Copied boston_sDNA1k_p_educivic.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_educivic
Copied boston_sDNA1k_p_leisure.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_leisure
Copied boston_sDNA1k_p_shopping.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_shopping
Copied boston_sDNA1k_p_work.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_sDNA1k_p_work
Copied boston_st_gd_AllPOIS.shp to Z:\heyutian\Kaunas_Boston\Boston\Boston.gdb\boston_st_gd_AllPOIS
Added new field nqpd1keduc in boston_sDNA1k_p_educivic
Copied data from NQPDE1000c to nqpd1keduc in boston_sDNA1k_p_educivic
Added new field nqpd1kleis in boston_sDNA1k_p_leisure
Copied data from NQPDE1000c to nqpd1kleis in boston_sDNA1k_p_leisure
Added new field nqpd1kshop in boston_sDNA1k_p_shopping
Copied data from NQPDE1000c to nqpd1kshop in boston_sDNA1k_p_shopping
Added new field nqpd1kwork in boston_sDNA1k_p_work
Copied data from NQPDE1000c to nqpd1kwork in boston_sD

RuntimeError: Object: Error in executing tool

#### Step 5: Export as txt file(s) for IBM SPSS correlation calculations